In [1]:
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/newCFG/hw-memory/benchmarks/spec_example/spec2006_470.lbm/lbm_part.asm"

In [2]:
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str)
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str, inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 0000000000400498
.plt 00000000004004b0
__libc_start_main@plt 00000000004004d0
__gmon_start__@plt 00000000004004e0
abort@plt 00000000004004f0
_start 0000000000400500
__wrap_main 0000000000400534
_dl_relocate_static_pie 0000000000400540
call_weak_fn 0000000000400544
deregister_tm_clones 0000000000400560
register_tm_clones 0000000000400590
__do_global_dtors_aux 00000000004005d0
frame_dummy 0000000000400600
compare 0000000000400604
main 0000000000400634
_fini 0000000000400658


In [3]:
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

TypeError: 'str' object is not callable

In [ ]:
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()

tcfg.build_loop_hrchy()
for l in tcfg.loops:
    #print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    #print(l.back_edge.src.name,l.back_edge.dst.name)
    l.back_edge.is_backEdge = True

#for n in tcfg.all_nodes:
#    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
#    print("{} {}".format(n.name, temp))

for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)


#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




n0 n2 1.0
n5 n1 1.0
n2 n4 0.5
n2 n3 0.5
n3 n5 0.5
n4 n5 0.5


In [ ]:
from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table

for i in lds_table:
    print(i.ins.name,i.is_sp,i.ins.tokens,i.final_addr)

rwproc = RWProc(lds_table)

for rwu in rwproc.rw_table:
    print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)

str False ('400640', 'b9001fe0', 'str', None, 'w0, [x34, #28]') 0
ldr True ('400644', 'b9401fe0', 'ldr', None, 'w0, [x33, #28]') 0
str False ('400608', 'b9000fe0', 'str', None, 'w0, [x35, #14]') 4321294
ldr False ('400614', 'b9400fe0', 'ldr', None, 'w0, [x33, #12]') 4321292
ldr True ('400620', 'b9400fe0', 'ldr', None, 'w0, [sp, #12]') 0
('400640', 'b9001fe0', 'str', None, 'w0, [x34, #28]') 0 0 RWType.Unknown
('400644', 'b9401fe0', 'ldr', None, 'w0, [x33, #28]') 0 0 RWType.Unknown
('400608', 'b9000fe0', 'str', None, 'w0, [x35, #14]') 7.0 4321294 RWType.Unknown
('400614', 'b9400fe0', 'ldr', None, 'w0, [x33, #12]') 10.0 4321292 RWType.Unknown
('400620', 'b9400fe0', 'ldr', None, 'w0, [sp, #12]') 0 0 RWType.Unknown
